In [1]:
#| default_exp overlay.main

In [2]:
#| export
import logging
from datetime import timedelta
from itertools import product
from threading import Thread
from time import sleep, time

import dash_bootstrap_components as dbc
import orjson
import pandas as pd
import plotly.express as px
from aifront.overlay.default import background_color, timeout_wrapper
from aifront.overlay.driving import (
    action_callback_args,
    action_overlay,
    progress_callback_args,
    progress_overlay,
)
from aifront.overlay.medal import medal_badges, pb_badge
from aifront.overlay.round import round_overlay
from aifront.overlay.session import session_overlay
from dash import Dash, Input, Output, State, dcc, html
from dash.exceptions import PreventUpdate
from dash_extensions import WebSocket
from furl import furl

## Layout

In [3]:
#| exporti
WebSocketMock = dcc.ConfirmDialog

def layout(color, size, border=False):
    websockets = [WebSocketMock(id=f"ws-{ws}") for ws in ["action", "step", "session", "cam"]]
    children = [
        html.Meta(httpEquiv="refresh", content=2 * 3600),
        dcc.Location(id="url", refresh=False),
        html.Div(websockets, id="websockets"),
        dcc.Store(data=color, id="color"),
        html.Canvas(
            style=size | {"border-collapse": "collapse", "filter": "blur(0.5em)"},
            id="cam", className="position-absolute z-n1 opacity-50",
        ),
        dbc.Container(
            "",
            id="map-name",
            class_name="pt-2 d-flex justify-content-center",
            fluid=True,
        ),
        html.Div(
            "",
            id="pb",
            className="position-absolute top-0 end-0 p-2 pt-1",
        ),
        html.Div(
            "",
            id="load-map",
            className="position-absolute top-0 p-2 pt-1",
        ),
        html.Div(
            action_overlay(color),
            className="position-absolute top-0",
        ),
        dbc.Container(
            progress_overlay(),
            class_name="position-absolute bottom-0 p-2",
            fluid=True,
        ),
        dbc.Container(
            "",
            id="medals",
            class_name="pt-1 d-flex justify-content-center",
            fluid=True,
        ),
        dbc.Container(
            "",
            id="end-ui",
            class_name="pt-3 h-75 d-flex justify-content-center",
            fluid=True,
        ),
    ]
    return dbc.Container(
        children,
        class_name="position-relative p-0" + (" border border-light" if border else ""),
        style=size | {"background-color": "transparent", "font-size": "1.4em"},
        fluid=True,
    )

In [4]:
#| export
cam_callback_args = [
    """
    function(message) {
        if(typeof message === 'undefined') {
            throw window.dash_clientside.PreventUpdate;
        }
        var data = JSON.parse(message.data);

        const specifier = "44015444025645045745055946075a46085c460a5d460b5e470d60470e6147106347116447136548146748166848176948186a481a6c481b6d481c6e481d6f481f70482071482173482374482475482576482677482878482979472a7a472c7a472d7b472e7c472f7d46307e46327e46337f463480453581453781453882443983443a83443b84433d84433e85423f854240864241864142874144874045884046883f47883f48893e49893e4a893e4c8a3d4d8a3d4e8a3c4f8a3c508b3b518b3b528b3a538b3a548c39558c39568c38588c38598c375a8c375b8d365c8d365d8d355e8d355f8d34608d34618d33628d33638d32648e32658e31668e31678e31688e30698e306a8e2f6b8e2f6c8e2e6d8e2e6e8e2e6f8e2d708e2d718e2c718e2c728e2c738e2b748e2b758e2a768e2a778e2a788e29798e297a8e297b8e287c8e287d8e277e8e277f8e27808e26818e26828e26828e25838e25848e25858e24868e24878e23888e23898e238a8d228b8d228c8d228d8d218e8d218f8d21908d21918c20928c20928c20938c1f948c1f958b1f968b1f978b1f988b1f998a1f9a8a1e9b8a1e9c891e9d891f9e891f9f881fa0881fa1881fa1871fa28720a38620a48621a58521a68522a78522a88423a98324aa8325ab8225ac8226ad8127ad8128ae8029af7f2ab07f2cb17e2db27d2eb37c2fb47c31b57b32b67a34b67935b77937b87838b9773aba763bbb753dbc743fbc7340bd7242be7144bf7046c06f48c16e4ac16d4cc26c4ec36b50c46a52c56954c56856c66758c7655ac8645cc8635ec96260ca6063cb5f65cb5e67cc5c69cd5b6ccd5a6ece5870cf5773d05675d05477d1537ad1517cd2507fd34e81d34d84d44b86d54989d5488bd6468ed64590d74393d74195d84098d83e9bd93c9dd93ba0da39a2da37a5db36a8db34aadc32addc30b0dd2fb2dd2db5de2bb8de29bade28bddf26c0df25c2df23c5e021c8e020cae11fcde11dd0e11cd2e21bd5e21ad8e219dae319dde318dfe318e2e418e5e419e7e419eae51aece51befe51cf1e51df4e61ef6e620f8e621fbe723fde725";
        var n = specifier.length / 6, colors = new Array(n), i = 0;
        while (i < n) colors[i] = "#" + specifier.slice(i * 6, ++i * 6);

        function viridis(t) {
            return colors[Math.max(0, Math.min(n - 1, Math.floor(t * n)))];
        }

        var canvas = document.getElementById('cam');
        var ctx = canvas.getContext('2d');
        
        // Clear the canvas
        ctx.clearRect(0, 0, canvas.width, canvas.height);

        // Example: Drawing a simple grid
        var numRows = data.length;
        var numCols = data[0].length;
        var cellWidth = canvas.width / numCols;
        var cellHeight = canvas.height / numRows;

        for (var i = 0; i < numRows; i++) {
            for (var j = 0; j < numCols; j++) {
                ctx.fillStyle = viridis(data[i][j]);
                ctx.fillRect(j * cellWidth, i * cellHeight, cellWidth, cellHeight);
            }
        }
        return window.dash_clientside.no_update;
    }
    """,
    Output('cam', 'children'),
    Input("ws-cam", "message"),
]

clear_cam_callback_args = [
    """
    function(hidden) {
        var canvas = document.getElementById('cam');
        var ctx = canvas.getContext('2d');
        
        // Clear the canvas
        ctx.clearRect(0, 0, canvas.width, canvas.height);

        return window.dash_clientside.no_update;
    }
    """,
    Output('cam', 'hidden'),
    Input("cam", "n_clicks"),
]

In [5]:
#| exporti
def map_name_overlay(name, session_state, color):
    percent = session_state["round"] / session_state["nb_episode"] * 100
    color_bg = color.replace("rgb(", "rgba(").replace(")", ", 0.75)")
    style = {
        "font-size": "1.1em",
        "background": f"linear-gradient(to right, {color} 0%,{color} {percent:.2f}%,{color_bg} {percent:.2f}%,{color_bg} 100%)",
    }
    return dbc.Badge(name, pill=True, style=style)


def load_map_overlay(name, color):
    return dbc.Badge(
        [
            dbc.Spinner(color=color, size="sm"),
            html.Span(f"Loading next map: {name}", className="ps-2"),
        ],
        pill=True,
        color=background_color,
    )

In [6]:
#| export
class EnvOverlay(Thread):
    def __init__(
        self,
        color=px.colors.qualitative.Vivid[0],
        size=dict(width=960, height=540),
        after_done_duration=5000,
        debug=False,
        border=False,
        port=8051,
    ):
        super().__init__()
        self.after_done_duration = after_done_duration
        self.debug = debug
        self.port = port
        self.app = Dash(
            external_stylesheets=[dbc.themes.DARKLY, dbc.icons.FONT_AWESOME],
        )

        @self.app.callback(
            Output("websockets", "children"),
            Output("color", "data"),
            Input("url", "href"),
            State("color", "data"),
        )
        def websockets(href, color):
            url = furl(href)
            env_name = url.path.segments[0]
            websockets_children = [
                WebSocket(url=f"ws://localhost:2013/ws/{env_name}-{ws}", id=f"ws-{ws}")
                for ws in ["action", "step", "session", "cam"]
            ]
            return [websockets_children, url.args.get("color", color)]

        self.app.enable_dev_tools(dev_tools_silence_routes_logging=None)
        self.app.layout = layout(color, size, border)
        self.app.clientside_callback(*action_callback_args, prevent_initial_call=True)
        self.app.clientside_callback(*progress_callback_args, prevent_initial_call=True)
        self.app.clientside_callback(*cam_callback_args, prevent_initial_call=True)
        self.app.clientside_callback(*clear_cam_callback_args, prevent_initial_call=True)

        @self.app.callback(
            Output("map-name", "children"),
            Output("pb", "children"),
            Output("load-map", "children"),
            Output("medals", "children"),
            Output("end-ui", "children"),
            Output("cam", "n_clicks"),
            Input("ws-session", "message"),
            State("color", "data"),
            State("map-name", "children"),
            State("pb", "children"),
            State("load-map", "children"),
            State("medals", "children"),
            State("end-ui", "children"),
            State("cam", "n_clicks"),
            prevent_initial_call=True,
        )
        def session_callback(message, color, map_name, pb, load_map, medals, end_ui, cam_toggle):
            if not message or "data" not in message:
                raise PreventUpdate
            data = orjson.loads(message["data"])

            if data["status"] == "before_load_map":
                load_map = load_map_overlay(data["map"]["sanitized_name"], color)

            if data["status"] in ["before_session", "after_reset", "after_done"]:
                map_name = map_name_overlay(
                    data["config"]["map"]["sanitized_name"],
                    data["state"]["session_scheduler"],
                    color,
                )
                pb = pb_badge(
                    data["config"]["map"]["medals"], data["stats"]["overall"]["best"]
                )

            if data["status"] == "before_session":
                load_map = ""
                medals = ""
                end_ui = ""
            if data["status"] == "after_done":
                load_map = ""
                medals = timeout_wrapper(
                    medal_badges(data["config"]["map"]["medals"]),
                    self.after_done_duration,
                )
                end_ui = timeout_wrapper(
                    round_overlay(data),
                    self.after_done_duration,
                )
            if data["status"] == "after_session":
                map_name = map_name_overlay(
                    data["config"]["map"]["sanitized_name"],
                    dict(round=1, nb_episode=1),
                    color,
                )
                pb = pb_badge(
                    data["config"]["map"]["medals"], data["stats"]["overall"]["best"]
                )
                load_map = ""
                medals = medal_badges(data["config"]["map"]["medals"])
                end_ui = session_overlay(data)
            return map_name, pb, load_map, medals, end_ui, int(not cam_toggle)


        self.start()

    def run(self):
        self.app.run_server(debug=self.debug, port=self.port, jupyter_mode="external")

In [7]:
# |hide
# |eval: false
from nbdev.doclinks import nbdev_export

nbdev_export()